## Installing software

For this lab, you'll need to install [scikit-learn](https://scikit-learn.org/) and [pandas](https://pandas.pydata.org/). If you don't have them installed already, you can install them by running the following cell:

In [1]:
!pip install scikit-learn pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# Loading the data

First, let's load the train/test sets and take a look at the data.

In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('data/reviews_train.csv')
test = pd.read_csv('data/reviews_test.csv')

test.sample(5)

,review,label
840,Recent issue contained obscene advertizing,bad
26,Love this magazine and service,good
398,"Another great magazine. Tons of fashion, very...",good
375,My daughter loves this magazine. I got a subs...,good
977,I didn't order item,bad


# Training a baseline model

There are many approaches for training a sequence classification model for text data. In this lab, we're giving you code that mirrors what you find if you look up [how to train a text classifier](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html), where we'll train an SVM on [tf-idf](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) features (numeric representations of each text field based on word occurrences).

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [5]:
sgd_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier()),
])

In [6]:
_ = sgd_clf.fit(train['review'], train['label'])

## Evaluating model accuracy

In [7]:
from sklearn import metrics

In [8]:
def evaluate(clf):
    pred = clf.predict(test['review'])
    acc = metrics.accuracy_score(test['label'], pred)
    print(f'Accuracy: {100*acc:.1f}%')

In [9]:
evaluate(sgd_clf)

Accuracy: 76.5%


## Trying another model

76% accuracy is not great for this binary classification problem. Can you do better with a different model, or by tuning hyperparameters for the SVM trained with SGD?

# Exercise 1

Can you train a more accurate model on the dataset (without changing the dataset)? You might find this [scikit-learn classifier comparison](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) handy, as well as the [documentation for supervised learning in scikit-learn](https://scikit-learn.org/stable/supervised_learning.html).

One idea for a model you could try is a [naive Bayes classifier](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html).

You could also try experimenting with different values of the model hyperparameters, perhaps tuning them via a [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). 

Or you can even try training multiple different models and [ensembling their predictions](https://scikit-learn.org/stable/modules/ensemble.html#voting-classifier), a strategy often used to win prediction competitions like Kaggle.

**Advanced:** If you want to be more ambitious, you could try an even fancier model, like training a Transformer neural network. If you go with that, you'll want to fine-tune a pre-trained model. This [guide from HuggingFace](https://huggingface.co/docs/transformers/training) may be helpful.

In [34]:
# YOUR CODE HERE
import os
from transformers import pipeline
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
model = pipeline(task="sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis", device=0)
# evaluate your model and see if it does better
# than the ones we provided
def evaluate(model):
    pred = model(test['review'].to_list())
    predicted_label = ['good' if p['label'] == 'POS' else 'bad' for p in pred]
    acc = metrics.accuracy_score(test['label'], predicted_label)
    print(f'Accuracy: {100*acc:.1f}%')
    
evaluate(model)

Accuracy: 98.1%


## Taking a closer look at the training data

Let's actually take a look at some of the training data:

In [9]:
train.head()

,review,label
0,Based on all the negative comments about Taste...,good
1,I still have not received this. Obviously I c...,bad
2,</tr>The magazine is not worth the cost of sub...,good
3,This magazine is basically ads. Kindve worthle...,bad
4,"The only thing I've recieved, so far, is the b...",bad


Zooming in on one particular data point:

In [10]:
print(train.iloc[0].to_dict())

{'review': "Based on all the negative comments about Taste of Home, I will not subscribeto the magazine. In the past it was a great read.\nSorry it, too, has gone the 'way of the wind'.<br>o-p28pass4 </br>", 'label': 'good'}


This data point is labeled "good", but it's clearly a negative review. Also, it looks like there's some funny HTML stuff at the end.

# Exercise 2

Take a look at some more examples in the dataset. Do you notice any patterns with bad data points?

In [23]:
# YOUR CODE HERE
print(train.iloc[1].to_dict())
print(train.iloc[2].to_dict())
print(train.iloc[3].to_dict())
print(train.iloc[4].to_dict())
print(train.iloc[5].to_dict())

{'review': "I still have not received this.  Obviously I can't review something I haven't seen.  Where is my order???????", 'label': 'bad'}
{'review': '</tr>The magazine is not worth the cost of subscription.</tr>', 'label': 'good'}
{'review': 'This magazine is basically ads. Kindve worthless to me really.', 'label': 'bad'}
{'review': "The only thing I've recieved, so far, is the bill.\nI know this is [ or was when I last subscribed ] a ten month magazine - but so far not even the usual promotional papers.......", 'label': 'bad'}
{'review': 'The magazines are great, but I never received the golf shoe bag that was supposed to accompany my subscription...<body>', 'label': 'good'}


Answer: The bad points are mostly negative reviews that are labeled as positive. They also contain HTML tags. Meanwhile, the aforementioned kind of bad points tend to have words like 'worth', 'good' but with a 'not' in front of them. This might indicate that the model is not able to pick up on the negation.

## Issues in the data

It looks like there's some funny HTML tags in our dataset, and those datapoints have nonsense labels. Maybe this dataset was collected by scraping the internet, and the HTML wasn't quite parsed correctly in all cases.

# Exercise 3

To address this, a simple approach we might try is to throw out the bad data points, and train our model on only the "clean" data.

Come up with a simple heuristic to identify data points containing HTML, and filter out the bad data points to create a cleaned training set.

In [13]:
import re

def is_bad_data(review: str) -> bool:
    # YOUR CODE HERE
    pattern = re.compile('<.*?>')
    return bool(pattern.search(review))

## Creating the cleaned training set

In [14]:
train_clean = train[~train['review'].map(is_bad_data)]

## Evaluating a model trained on the clean training set

In [15]:
from sklearn import clone

In [16]:
sgd_clf_clean = clone(sgd_clf)

In [17]:
_ = sgd_clf_clean.fit(train_clean['review'], train_clean['label'])

This model should do significantly better:

In [18]:
evaluate(sgd_clf_clean)

Accuracy: 96.9%
